In [1]:
pip install langchain==0.2

  Using cached dataclasses_json-0.6.7-py3-none-any.whl.metadata (25 kB)
  Using cached typing_inspect-0.9.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached mypy_extensions-1.0.0-py3-none-any.whl.metadata (1.1 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.7/973.7 kB 829.2 kB/s eta 0:00:00MB/s eta 0:00:01:01
Using cached dataclasses_json-0.6.7-py3-none-any.whl (28 kB)
Using cached typing_inspect-0.9.0-py3-none-any.whl (8.8 kB)
Using cached mypy_extensions-1.0.0-py3-none-any.whl (4.7 kB)
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.12
    Uninstalling langchain-core-0.3.12:
      Successfully uninstalled langchain-core-0.3.12
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.3.0
    Uninstalling langchain-text-splitters-0

In [2]:
pip install dsm_services -U

Note: you may need to restart the kernel to use updated packages.


In [3]:
import dsm_services
model = dsm_services.llm.LLM(
    model_name='llama3.1',
    api_key="H9hWk3vo.iWZcjb3Kg8Cq57j2iT0QtjJG8LDbqwbU"
)

In [4]:
from dsm_services.llm.langchain import llmChatModel
llm = llmChatModel(
    model_name="llama3.1:70b", 
    api_key="H9hWk3vo.iWZcjb3Kg8Cq57j2iT0QtjJG8LDbqwbU"
)

In [18]:
pip freeze | grep dsm

dsm_services==0.0.10
Note: you may need to restart the kernel to use updated packages.


In [19]:
pip freeze | grep langchain

langchain==0.3.4
langchain-core==0.3.12
langchain-text-splitters==0.3.0
Note: you may need to restart the kernel to use updated packages.


In [5]:
from typing import List, Dict

from langchain.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field, validator

from langchain.prompts import PromptTemplate
from langchain.schema import BaseOutputParser
from pydantic import BaseModel, ValidationError
from typing import Any, List

In [6]:
import time
from functools import wraps

def retry(n):
    """
    A decorator to retry a function n times upon failure, and print the number of attempts made.
    
    Parameters:
    -----------
    n : int
        The number of retry attempts.
    
    Returns:
    --------
    Callable
        A decorator that retries the decorated function up to n times.
    """
    def decorator_retry(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            attempts = 0
            while attempts < n:
                try:
                    attempts += 1
                    # Pass the attempt count to the decorated function
                    result = func(*args, attempt_number=attempts, **kwargs)
                    print(f"Success on attempt {attempts}")
                    return result
                except Exception as e:
                    print(f"Attempt {attempts} failed: {e}")
                    if attempts == n:
                        raise  # Re-raise the exception if it's the last attempt
                    # time.sleep(2)  # Optional delay between retries
        return wrapper
    return decorator_retry

In [7]:
class Entity(BaseModel):
    """
    An entity details in the knowledge graph. The entity detials consist of name, class and description.

    Attributes:
    -----------
    name : str
        The human-readable name of the entity.
    class : str
        The classification or type of the entity (e.g., Person, Product, etc.).
    description : str
        A brief description of the entity.
    """
    entity_name: str
    entity_class: str
    description: str

class Relation(BaseModel):
    """
    A class to represent a relationship (triple) in the knowledge graph.

    Attributes:
    -----------
    head : Entity
        The subject or head entity in the relationship.
    predicate : str
        The relationship or predicate that links the head and tail (e.g., Person, Product, etc.).
    tail : Entity
        The object or tail entity in the relationship.
    """
    head: Entity
    predicate: str
    tail: Entity

In [9]:
import pandas as pd

In [10]:
# Define a custom Output Parser
# class EntityRelationOutputParser(BaseOutputParser):
#     def parse(self, text: str) -> List[Relation]:
#         try:
#             # You can implement custom parsing logic here, for example, splitting or extracting structured data
#             parsed_data = eval(text)  # Assuming LLM responds in Python dictionary format
#             relations = [Relation(**relation) for relation in parsed_data]
#             return relations
#         except ValidationError as e:
#             raise ValueError(f"Parsing error: {e}")

class EntityRelationOutputParser(BaseOutputParser):
    def parse(self, text: str) -> Dict[str, pd.DataFrame]:
        try:
            # Parse the response assuming it comes in JSON format
            parsed_data = eval(text)  # Assuming LLM responds in Python dictionary format

            # Extract relations
            relations = [Relation(**relation) for relation in parsed_data]

            # Create unique entity records for head and tail
            entities = {}
            for relation in relations:
                entities[relation.head.entity_name] = relation.head
                entities[relation.tail.entity_name] = relation.tail

            # Create DataFrames
            entity_df = pd.DataFrame([entity.dict() for entity in entities.values()])
            relations_df = pd.DataFrame([{
                'head_name': relation.head.entity_name,
                'head_class': relation.head.entity_class,
                'head_description': relation.head.description,
                'predicate': relation.predicate,
                'tail_name': relation.tail.entity_name,
                'tail_class': relation.tail.entity_class,
                'tail_description': relation.tail.description
            } for relation in relations])

            return {
                "entity_df": entity_df,
                "relations_df": relations_df
            }

        except ValidationError as e:
            raise ValueError(f"Parsing error: {e}")

# Define your prompt template
prompt_template = """
Please extract the entities and relations from the following Wikipedia text to the database.
Ensure that the entities and relationships are accurate.
Given the following input, return the entities and relations as JSON format.

ps. reponse only json data

Context: {context}

Input: {input_text}

Format your output as follows:
[
    {{
        "head": {{
            "entity_name": "name of the head entity",
            "entity_class": "classification of head",
            "description": "brief description"
        }},
        "predicate": "relationship description",
        "tail": {{
            "entity_name": "name of the tail entity",
            "entity_class": "classification of tail",
            "description": "brief description"
        }}
    }}
]

Response:
"""

# Instantiate a PromptTemplate
prompt = PromptTemplate(
    input_variables=["input_text", "context"],
    template=prompt_template
)

# Function to create the full prompt and parse output
@retry(5)
def create_and_parse_output(input_text: str, context:str, llm, attempt_number) -> List[Relation]:
    # Create the full prompt
    final_prompt = prompt.format(input_text=input_text, context=context)
    
    # Call the LLM (e.g., OpenAI, Ollama) with the prompt
    response = llm.invoke(final_prompt).content  # This depends on your LLM client

    # Parse the output using the custom parser
    parser = EntityRelationOutputParser()
    output = parser.parse(response)
    output.update({
        'attempt_number': attempt_number
    })
    return output

In [11]:
df = pd.read_parquet('./91_chunk.parquet')
df

,wiki_id,url,chunks,chunk_id
0,Q43436,https://en.wikipedia.org/wiki/Pearl,Pearl\n,1
1,Q43436,https://en.wikipedia.org/wiki/Pearl,"A pearl is a hard, glistening object produced ...",2
2,Q43436,https://en.wikipedia.org/wiki/Pearl,The most valuable pearls occur spontaneously i...,3
3,Q43436,https://en.wikipedia.org/wiki/Pearl,"Whether wild or cultured, gem-quality pearls a...",4
4,Q43436,https://en.wikipedia.org/wiki/Pearl,\nThe English word pearl comes from the French...,5
...,...,...,...,...
97,Q43088,https://en.wikipedia.org/wiki/Ruby,"In 1903, Verneuil announced he could produce s...",25
98,Q43088,https://en.wikipedia.org/wiki/Ruby,Other processes in which synthetic rubies can ...,26
99,Q43088,https://en.wikipedia.org/wiki/Ruby,\nSynthetic rubies have technological uses as ...,27
100,Q43088,https://en.wikipedia.org/wiki/Ruby,Rubies are also used in applications where hig...,28


In [12]:
output = create_and_parse_output(input_text=df.iloc[0].chunks, llm=llm, context=df.iloc[0].url)
output

Success on attempt 1


{'entity_df':       entity_name      entity_class  \
 0           Pearl           Mineral   
 1  Oriental Pearl  Alternative Name   
 2         Mollusk   Living Organism   
 3          Luxury           Concept   
 4          Wisdom           Concept   
 
                                          description  
 0  A pearl is a hard, glistening object produced ...  
 1                      An alternative name for pearl  
 2  A type of invertebrate animal that produces pe...  
 3  A state of great comfort or elegance, especial...  
 4  The ability to discern or judge what is true, ...  ,
 'relations_df':   head_name head_class                                   head_description  \
 0     Pearl    Mineral  A pearl is a hard, glistening object produced ...   
 1     Pearl    Mineral  A pearl is a hard, glistening object produced ...   
 2     Pearl    Mineral  A pearl is a hard, glistening object produced ...   
 3     Pearl    Mineral  A pearl is a hard, glistening object produced ...   
 


In [13]:
entity_df = output['entity_df']
entity_df

,entity_name,entity_class,description
0,Pearl,Mineral,"A pearl is a hard, glistening object produced ..."
1,Oriental Pearl,Alternative Name,An alternative name for pearl
2,Mollusk,Living Organism,A type of invertebrate animal that produces pe...
3,Luxury,Concept,"A state of great comfort or elegance, especial..."
4,Wisdom,Concept,"The ability to discern or judge what is true, ..."


In [14]:
relations_df = output['relations_df'][['head_name', 'predicate', 'tail_name']]
relations_df

,head_name,predicate,tail_name
0,Pearl,is also known as,Oriental Pearl
1,Pearl,is formed by,Mollusk
2,Pearl,is often associated with,Luxury
3,Pearl,has a symbolic meaning of,Wisdom


In [15]:
relations_df\
    .merge(right=entity_df, left_on='head_name', right_on='entity_name', how='left')\
    .merge(right=entity_df, left_on='tail_name', right_on='entity_name', how='left')

,head_name,predicate,tail_name,entity_name_x,entity_class_x,description_x,entity_name_y,entity_class_y,description_y
0,Pearl,is also known as,Oriental Pearl,Pearl,Mineral,"A pearl is a hard, glistening object produced ...",Oriental Pearl,Alternative Name,An alternative name for pearl
1,Pearl,is formed by,Mollusk,Pearl,Mineral,"A pearl is a hard, glistening object produced ...",Mollusk,Living Organism,A type of invertebrate animal that produces pe...
2,Pearl,is often associated with,Luxury,Pearl,Mineral,"A pearl is a hard, glistening object produced ...",Luxury,Concept,"A state of great comfort or elegance, especial..."
3,Pearl,has a symbolic meaning of,Wisdom,Pearl,Mineral,"A pearl is a hard, glistening object produced ...",Wisdom,Concept,"The ability to discern or judge what is true, ..."
